---
## Python Packages & Directories
---

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from __future__ import (division, print_function, absolute_import,
                        unicode_literals)
import  sys, os, glob, time, IPython

import astropy.constants as const
import astropy.units as u
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
# from astropy.utils.data import get_pkg_data_filename
from astropy.coordinates import SkyCoord, EarthLocation

# from PyAstronomy import pyasl

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import AutoMinorLocator
plt.rcParams["font.family"] = "Times New Roman"

import numpy as np
import scipy as sp
import pandas as pd

import seaborn as sns
sns.set_palette("colorblind")
colors = sns.color_palette("colorblind", 10) + sns.color_palette("Paired", 10)[2:] + sns.color_palette("Set2", 10)

# from smh import Session

import spag.read_data as rd
import spag.utils as spagu
from spag.convert import *
from spag.coordinates import *

script_dir = "/".join(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"].split("/")[:-1]) + "/"
# script_dir = os.path.dirname(os.path.realpath(__file__))+"/"

        Use @ instead. [gala.coordinates.sgr]
        Use @ instead. [gala.coordinates.orphan]
        Use @ instead. [gala.coordinates.magellanic_stream]


In [2]:
## Show all columns and rows of the dataframe
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows

---
## Read-in Grevesse 1998 Solar Abundances
---

In [6]:
grevesse1998_solar_abundances.columns

Index(['Z', 'elem', 'photosphere_logeps', 'photosphere_logeps_err',
       'meteorite_logeps', 'meteorite_logeps_err',
       'photosphere-meteorite_logeps_diff'],
      dtype='object')

In [ ]:
grevesse1998_solar_abundances = pd.read_csv('/Users/ayelland/Research/metal-poor-stars/spag/spag/data/solar/grevesse1998_solarabundance_table1.csv')

solar_abund = dict()
for index, row in grevesse1998_solar_abundances.iterrows():
    if pd.notna(row['photosphere_logeps']):
        if row['Z'] == 92:
            solar_abund[row['elem']] = float(row['photosphere_logeps'].replace('<', ''))
        else:
            solar_abund[row['elem']] = float(row['photosphere_logeps'])
    elif pd.notna(row['meteorite_logeps']):
        solar_abund[row['elem']] = float(row['meteorite_logeps'])
    else:
        solar_abund[row['elem']] = np.nan

display(solar_abund)

{'H': 12.0,
 'He': 10.93,
 'Li': 1.1,
 'Be': 1.4,
 'B': 2.55,
 'C': 8.52,
 'N': 7.92,
 'O': 8.83,
 'F': 4.56,
 'Ne': 8.08,
 'Na': 6.33,
 'Mg': 7.58,
 'Al': 6.47,
 'Si': 7.55,
 'P': 5.45,
 'S': 7.33,
 'Cl': 5.5,
 'Ar': 6.4,
 'K': 5.12,
 'Ca': 6.36,
 'Sc': 3.17,
 'Ti': 5.02,
 'V': 4.0,
 'Cr': 5.67,
 'Mn': 5.39,
 'Fe': 7.5,
 'Co': 4.92,
 'Ni': 6.25,
 'Cu': 4.21,
 'Zn': 4.6,
 'Ga': 2.88,
 'Ge': 3.41,
 'As': 2.37,
 'Se': 3.41,
 'Br': 2.63,
 'Kr': 3.31,
 'Rb': 2.6,
 'Sr': 2.97,
 'Y': 2.24,
 'Zr': 2.6,
 'Nb': 1.42,
 'Mo': 1.92,
 'Ru': 1.84,
 'Rh': 1.12,
 'Pd': 1.69,
 'Ag': 0.94,
 'Cd': 1.77,
 'In': 1.66,
 'Sn': 2.0,
 'Sb': 1.0,
 'Te': 2.24,
 'I': 1.51,
 'Xe': 2.17,
 'Cs': 1.13,
 'Ba': 2.13,
 'La': 1.17,
 'Ce': 1.58,
 'Pr': 0.71,
 'Nd': 1.5,
 'Sm': 1.01,
 'Eu': 0.51,
 'Gd': 1.12,
 'Tb': -0.1,
 'Dy': 1.14,
 'Ho': 0.26,
 'Er': 0.93,
 'Tm': 0.0,
 'Yb': 1.08,
 'Lu': 0.06,
 'Hf': 0.88,
 'Ta': -0.13,
 'W': 1.11,
 'Re': 0.28,
 'Os': 1.45,
 'Ir': 1.35,
 'Pt': 1.8,
 'Au': 1.01,
 'Hg': 1.13,
 'Tl': 0.9,

---
## Calulate the logeps values from the [X/Fe] values
---

In [ ]:
lai2008 = pd.read_csv(script_dir + 'table10.csv', comment='#')

for i, row in lai2008.iterrows():
    
    ## Convert [Fe/H] to epsFe
    epsFe = row['[Fe/H]'] + solar_abund['Fe']
    lai2008.at[i, 'Fe'] = epsFe

    ## Convert [X/Fe] to epsX
    for col in row.index:
        if col.startswith('[') and col.endswith('/Fe]'):
            elem = col.split('[')[1].split('/')[0]
            if elem.endswith('II'):
                elem = elem[:-2]

            feh = float(row['[Fe/H]'])
            if isinstance(row[col], str):
                if '<' in row[col]:
                    xfe = float(row[col].replace('<', ''))
                else:
                    xfe = float(row[col])
            elif isinstance(row[col], float):
                xfe = row[col]
            else:
                print(type(row[col]), row[col])
                raise ValueError(f"Unexpected value for {col}: {row[col]}")

            epsX = xfe + feh + solar_abund[elem]
            lai2008.at[i, elem] = epsX

## Drop the original columns
lai2008 = lai2008.drop(columns=[col for col in lai2008.columns if col.startswith('[') and col.endswith('/Fe]')])
lai2008 = lai2008.drop(columns=[col for col in lai2008.columns if col.startswith('e_[') and col.endswith('/Fe]')])
lai2008 = lai2008.drop(columns=[col for col in lai2008.columns if col.startswith('[') and col.endswith('/H]')])
lai2008 = lai2008.drop(columns=[col for col in lai2008.columns if col.startswith('e_[') and col.endswith('/H]')])

## Round all values to 2 decimal places
lai2008 = lai2008.round(2)

## Re-order columns
column_order = ['Star', 'Li', 'Be', 'C', 'N', 'O', 'F', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'Sn', 'Te', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'W', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Pb', 'Bi', 'Th', 'U', 'CaII', 'TiII', 'VII', 'CrII', 'MnII', 'FeII']
for col in column_order:
    if col not in lai2008.columns:
        lai2008[col] = np.nan
lai2008 = lai2008[column_order]

## Reorder the rows y 'Star'
row_order = [
    'CS 22957-022',
    'CS 22963-004',
    'CS 31078-018',
    'BS 16928-053',
    'BS 16929-005',
    'CS 30325-028',
    'CS 30312-059',
    'CS 22884-108',
    'BS 16084-160',
    'BS 16080-054',
    'BS 16080-093',
    'CS 30336-049',
    'CS 22944-032',
    'CS 29502-092',
    'BD+03 740',
    'BD+23 3130',
    'BD+24 1676',
    'BS 16077-007',
    'CS 22872-102',
    'CS 22878-027',
    'CS 22880-086',
    'CS 22965-054',
    'CS 29506-007',
    'CS 29522-046',
    'CS 31085-024',
    'BS 16467-062',
    'BS 16550-087',
    'BS 16545-089',
]
lai2008 = lai2008.set_index('Star')
lai2008 = lai2008.reindex(row_order)
lai2008 = lai2008.reset_index()

## Display the updated DataFrame
display(lai2008)

lai2008.to_csv(script_dir + 'logeps_abundances.csv', index=False)

,Star,Li,Be,C,N,O,F,Na,Mg,Al,Si,P,S,K,Ca,Sc,Ti,V,Cr,Mn,Fe,Co,Ni,Cu,Zn,Ga,Ge,As,Se,Rb,Sr,Y,Zr,Nb,Mo,Ru,Rh,Pd,Ag,Cd,Sn,Te,Ba,La,Ce,Pr,Nd,Sm,Eu,Gd,Tb,Dy,Ho,Er,Tm,Yb,Lu,Hf,W,Os,Ir,Pt,Au,Hg,Pb,Bi,Th,U,CaII,TiII,VII,CrII,MnII,FeII
0,CS 22957-022,NaN,NaN,5.75,5.20,6.50,NaN,NaN,4.93,NaN,NaN,NaN,NaN,NaN,3.65,0.29,2.42,1.11,NaN,2.21,4.57,2.28,3.30,0.05,1.98,NaN,NaN,NaN,NaN,NaN,-0.37,-1.06,-0.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.81,NaN,NaN,NaN,NaN,NaN,-2.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CS 22963-004,NaN,NaN,5.50,5.30,6.40,NaN,NaN,4.51,2.20,4.55,NaN,NaN,NaN,3.25,0.02,2.04,0.65,NaN,1.55,4.08,1.98,2.91,-0.15,NaN,NaN,NaN,NaN,NaN,NaN,-1.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.73,NaN,NaN,NaN,NaN,NaN,-1.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CS 31078-018,NaN,NaN,6.05,4.70,6.80,NaN,NaN,5.16,2.90,NaN,NaN,NaN,NaN,3.93,0.43,2.56,1.29,2.65,2.56,4.66,2.42,3.50,0.35,2.17,NaN,NaN,NaN,NaN,NaN,0.37,-0.36,0.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.01,-0.93,NaN,NaN,NaN,NaN,-1.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BS 16928-053,NaN,NaN,5.20,5.90,NaN,NaN,3.29,4.80,2.50,5.64,NaN,NaN,NaN,3.50,-0.06,2.16,0.56,NaN,1.82,4.43,1.89,3.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.44,-1.24,-0.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.76,NaN,NaN,NaN,NaN,NaN,-2.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BS 16929-005,NaN,NaN,6.15,4.90,NaN,NaN,2.98,4.69,2.47,NaN,NaN,NaN,NaN,3.35,-0.20,2.18,0.67,NaN,NaN,4.16,1.96,2.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,-0.88,-0.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.49,NaN,NaN,NaN,NaN,NaN,-2.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,CS 30325-028,NaN,NaN,6.00,4.80,6.50,NaN,NaN,5.09,NaN,NaN,NaN,NaN,NaN,3.78,0.35,2.47,1.13,2.76,2.08,4.60,2.25,3.31,0.13,2.03,NaN,NaN,NaN,NaN,NaN,0.23,-0.65,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.11,-1.91,NaN,NaN,NaN,NaN,-2.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,CS 30312-059,NaN,NaN,5.65,4.30,NaN,NaN,3.36,4.85,2.45,NaN,NaN,NaN,NaN,3.55,0.10,2.27,0.90,2.62,1.72,4.36,2.06,3.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.19,-0.91,-0.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.02,NaN,NaN,NaN,NaN,NaN,-2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,CS 22884-108,NaN,NaN,6.30,6.00,NaN,NaN,NaN,4.62,2.60,4.52,NaN,NaN,NaN,3.58,0.48,2.68,NaN,NaN,NaN,4.36,NaN,3.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.77,NaN,NaN,NaN,NaN,NaN,-0.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,BS 16084-160,NaN,NaN,5.20,5.50,5.90,NaN,2.91,4.59,2.30,4.97,NaN,NaN,NaN,3.34,0.06,1.91,0.66,NaN,1.75,4.30,1.81,2.92,-0.63,NaN,NaN,NaN,NaN,NaN,NaN,-2.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.06,NaN,NaN,NaN,NaN,NaN,-2.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,BS 16080-054,NaN,NaN,5.00,5.60,NaN,NaN,3.74,4.95,2.70,4.95,NaN,NaN,NaN,3.70,0.08,2.25,0.90,2.39,1.73,4.43,1.93,3.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.06,-0.71,-0.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.36,NaN,NaN,NaN,NaN,NaN,-2.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
